In [25]:
!pip install efficientnet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 KB 7.9 MB/s eta 0:00:00


In [49]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras.layers import Conv2D, Dense, Dropout, Flatten, GlobalAveragePooling2D
import numpy as np
from efficientnet.keras import EfficientNetB0
from google.colab import drive
import os

drive.mount('/content/drive')
# 現在のディレクトリを取得する
current_dir = os.getcwd()

# ディレクトリ内のファイルとフォルダーを表示する
for entry in os.scandir(current_dir):
    if entry.is_file():
        print('File:', entry.name)
    elif entry.is_dir():
        print('Directory:', entry.name)

Mounted at /content/drive
Directory: .config
Directory: drive
Directory: sample_data


In [62]:
#数値設定
num_class = 4

In [19]:
model = keras.Sequential([
    #tf.keras.layers.Flatten(input_shape=(32, 32, 3)),
    keras.EfficientNetB0(include_top=False, weights='imagenet', input_shape=(224,224,3))
    #keras.layers.Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5)
    Dense(num_classes, activation='softmax')
])

SyntaxError: ignored

In [63]:
# EfficientNetB0の事前学習済み重みをロードする
base_model = EfficientNetB0(include_top=False, weights='imagenet', input_shape=(32,32,3))
# Sequentialモデルを定義
model = keras.Sequential()
# EfficientNetB0の重みを凍結
base_model.trainable = False
# EfficientNetB0をSequentialモデルに追加
model.add(base_model)
# Flatten層を追加して、EfficientNetB0の出力を1次元化する
#model.add(Flatten())
model.add(GlobalAveragePooling2D())
model.add(Dense(num_class, activation='softmax'))
# Dense層を追加
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
# 最終出力層を追加
model.add(Dense(num_class, activation='softmax'))

In [64]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [65]:
#データ振り分け
# データセットを読み込む
x_train = np.load('/content/drive/MyDrive/datasets/ndarray/xtrain_1.npy')
y_train = np.load('/content/drive/MyDrive/datasets/ndarray/ytrain_1.npy')

# データを正規化する
x_train = x_train.astype('float32') / 255.

# 入力データと正解ラベルをランダムにシャッフルする
np.random.seed(0)
train_shuffle = np.random.permutation(len(x_train))
x_train, y_train = x_train[train_shuffle], y_train[train_shuffle]

# データを学習用とテスト用に分割する
train_size = int(0.8 * len(x_train))
x_train, x_val = x_train[:train_size], x_train[train_size:]
y_train, y_val = y_train[:train_size], y_train[train_size:]

print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print('x_val shape:', x_test.shape)
print('y_val shape:', y_test.shape)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
y_train = keras.utils.to_categorical(y_train, num_class)

x_train shape: (31, 32, 32, 3)
y_train shape: (31, 1)
x_val shape: (10000, 32, 32, 3)
y_val shape: (10000, 1)


In [44]:
#(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
y_train = keras.utils.to_categorical(y_train, num_class)

print(np.shape(x_train))
print(np.shape(x_test))
print(np.shape(y_train))
print(np.shape(y_test))

(50000, 32, 32, 3)
(10000, 32, 32, 3)
(50000, 100)
(10000, 1)


In [38]:
print(y_train)

[[19]
 [29]
 [ 0]
 ...
 [ 3]
 [ 7]
 [73]]


In [66]:
model.fit(x_train, y_train, epochs=10, batch_size=256)

Epoch 1/10
1/1 [==============================] - 6s 6s/step - loss: 1.4053 - accuracy: 0.2581
Epoch 2/10
1/1 [==============================] - 0s 20ms/step - loss: 1.3889 - accuracy: 0.2903
Epoch 3/10
1/1 [==============================] - 0s 20ms/step - loss: 1.3752 - accuracy: 0.2581
Epoch 4/10
1/1 [==============================] - 0s 19ms/step - loss: 1.3344 - accuracy: 0.5484
Epoch 5/10
1/1 [==============================] - 0s 18ms/step - loss: 1.2937 - accuracy: 0.8387
Epoch 6/10
1/1 [==============================] - 0s 17ms/step - loss: 1.2824 - accuracy: 0.8710
Epoch 7/10
1/1 [==============================] - 0s 19ms/step - loss: 1.2372 - accuracy: 0.9677
Epoch 8/10
1/1 [==============================] - 0s 17ms/step - loss: 1.2118 - accuracy: 1.0000
Epoch 9/10
1/1 [==============================] - 0s 18ms/step - loss: 1.2063 - accuracy: 1.0000
Epoch 10/10
1/1 [==============================] - 0s 18ms/step - loss: 1.1609 - accuracy: 1.0000


In [54]:
test_loss, test_acc = model.evaluate(x_test, y_test)

ValueError: ignored

In [67]:
from PIL import Image
import os
import cv2

# 画像ファイルを開く
img = Image.open('/content/drive/MyDrive/pro/test.png')

# 32x32にリサイズする
img = img.resize((32, 32))

# リサイズ後の画像を保存する
img.save('/content/drive/MyDrive/pro/test2.jpg')

img = cv2.imread("/content/drive/MyDrive/pro/test2.jpg")
img_processed = img.astype(np.float32) / 255.0
img_processed = np.expand_dims(img_processed, axis=0)
hoge = model.predict(img_processed)
print(hoge)

1/1 [==============================] - 1s 1s/step
[[0.3232453  0.2234607  0.21701042 0.23628362]]
